100本ノック 第5章

問４０

In [2]:
#ai.ja.zipをインストールし、解凍する
!wget https://nlp100.github.io/data/ai.ja.zip
!unzip ai.ja.zip

# MeCabをインストール（CaboChaの実行に必要）
!apt install mecab libmecab-dev mecab-ipadic-utf8

# CRF++をインストール（CaboChaの実行に必要）
FILE_ID = "0B4y35FiV1wh7QVR6VXJ5dWExSTQ"
FILE_NAME = "crfpp.tar.gz"
!wget 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O $FILE_NAME
!tar xvf crfpp.tar.gz
%cd CRF++-0.58
!./configure && make && make install && ldconfig
%cd ..

# CaboChaをインストール
FILE_ID = "0B4y35FiV1wh7SDd1Q1dUQkZQaUU"
FILE_NAME = "cabocha-0.69.tar.bz2"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt
!tar -xvf cabocha-0.69.tar.bz2
%cd cabocha-0.69
!./configure -with-charset=utf-8 && make && make check && make install && ldconfig
%cd ..

#係り受け解析結果をai.ja.txt.parsedに保存
!cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt

--2023-12-08 05:44:18--  https://nlp100.github.io/data/ai.ja.zip
Resolving nlp100.github.io (nlp100.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17516 (17K) [application/zip]
Saving to: ‘ai.ja.zip’

ai.ja.zip           100%[===================>]  17.11K  --.-KB/s    in 0s      

2023-12-08 05:44:19 (113 MB/s) - ‘ai.ja.zip’ saved [17516/17516]

Archive:  ai.ja.zip
  inflating: ai.ja.txt               
  inflating: readme.ai.ja.md         
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
mecab is already the newest version (0.996-14build9).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
--2023-12-08 05:44:21--  https://docs.google.

In [5]:
class Morph:
  def __init__(self, line):
    surface, other = line.split("\t")
    other = other.split(",")
    self.surface = surface
    self.base = other[-3]
    self.pos = other[0]
    self.pos1 = other[1]

sentences = [] #文リスト
morphs = [] #形態素リスト

with open("./ai.ja.txt.parsed") as f:
  for line in f:
    if line[0] == "*":
      continue
    elif line != "EOS\n":
      morphs.append(Morph(line))
    else:  #EOS（文末）の場合
      sentences.append(morphs)
      morphs = []

for i in sentences[0]:
    print(vars(i))

FileNotFoundError: ignored

問４１

In [4]:
class Sentence:
  def __init__(self, chunks):
    self.chunks = chunks
    for i, chunk in enumerate(self.chunks):
      if chunk.dst not in [None, -1]:
        self.chunks[chunk.dst].srcs.append(i)

class Chunk:
  def __init__(self, morphs, dst, chunk_id):
    self.morphs = morphs
    self.dst = dst
    self.srcs = []
    self.chunk_id = chunk_id

class Morph:
  def __init__(self, line):
    surface, other = line.split("\t")
    other = other.split(",")
    self.surface = surface
    self.base = other[-3]
    self.pos = other[0]
    self.pos1 = other[1]

sentences = [] #文リスト
chunks = [] #節リスト
morphs = [] #形態素リスト
chunk_id = 0 #文節番号

with open("./ai.ja.txt.parsed") as f:
  for line in f:
    if line[0] == "*":
      if morphs:
        chunks.append(Chunk(morphs, dst, chunk_id))
        chunk_id += 1
        morphs = []
      dst = int(line.split()[2].replace("D", ""))
    elif line != "EOS\n":
      morphs.append(Morph(line))
    else:
      chunks.append(Chunk(morphs, dst, chunk_id))
      sentences.append(Sentence(chunks))

      morphs = []
      chunks = []
      dst = None
      chunk_id = 0

for chunk in sentences[2].chunks:
  chunk_str = "".join([morph.surface for morph in chunk.morphs])
  print(f"文節の文字列：{chunk_str}\n係り先の文節番号：{chunk.dst}\n")

FileNotFoundError: ignored

問４２

In [ ]:
for chunk in sentences[2].chunks:
  if int(chunk.dst) == -1:
    continue
  else:
    surf = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
    next_surf = "".join([morph.surface for morph in sentences[2].chunks[int(chunk.dst)].morphs if morph.pos != "記号"])
    print(f"{surf}\t{next_surf}")

問４３

In [ ]:
for chunk in sentences[2].chunks:
  if int(chunk.dst) == -1:
    continue
  else:
    surf = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
    next_surf = "".join([morph.surface for morph in sentences[2].chunks[int(chunk.dst)].morphs if morph.pos != "記号"])
    pos_noun = [morph.surface for morph in chunk.morphs if morph.pos == "名詞"]
    pos_verb = [morph.surface for morph in sentences[2].chunks[int(chunk.dst)].morphs if morph.pos == "動詞"]
    if pos_noun and pos_verb:
      print(f"{surf}\t{next_surf}")

問４４

In [ ]:
#pydotをインストール
!pip install pydot
#日本語フォントをインストール
!apt install fonts-ipafont-gothic

import pydot_ng as pydot

pair = []
for chunk in sentences[2].chunks:
  if int(chunk.dst) == -1:
    continue
  else:
    surf = "".join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
    next_surf = "".join([morph.surface for morph in sentences[2].chunks[int(chunk.dst)].morphs if morph.pos != "記号"]) #文節のリストに係り先番号をindexに指定。その文節の形態素リストを取得
    pair.append((surf, next_surf))

img = pydot.Dot()
img.set_node_defaults(fontname="MS Mincho")
for s, t in pair:
  img.add_edge(pydot.Edge(s, t))
img.write_png("./result44.png")

問４５

In [ ]:
with open("./result45.txt", "w") as f:
  for i in range(len(sentences)):
    for chunk in sentences[i].chunks:
      for morph in chunk.morphs:
        if morph.pos == "動詞":
          particles = []
          for src in chunk.srcs:
            particles += [morph.surface for morph in sentences[i].chunks[src].morphs if morph.pos == "助詞"]
          if len(particles) > 1:
            particles = set(particles)
            particles = sorted(list(particles))
            form = " ".join(particles)
            print(f"{morph.base}\t{form}", file=f)

In [ ]:
#コーパス中で頻出する述語と格パターンの組み合わせ
!cat ./result45.txt | sort | uniq -c | sort -nr | head -n 5
#「行う」「なる」「与える」という動詞の格パターン
!cat ./result45.txt |grep "行う" | sort |uniq -c | sort -nr |head -n 5
!cat ./result45.txt |grep "なる" | sort |uniq -c | sort -nr |head -n 5
!cat ./result45.txt |grep "与える" | sort |uniq -c | sort -nr |head -n 5

問４６

In [ ]:
with open("./result46.txt", "w") as f:
  for i in range(len(sentences)):
    for chunk in sentences[i].chunks:
      for morph in chunk.morphs:
        if morph.pos == "動詞":
          particles = []
          items = []
          for src in chunk.srcs:
            particles += [morph.surface for morph in sentences[i].chunks[src].morphs if morph.pos == "助詞"]
            items += ["".join([morph.surface for morph in sentences[i].chunks[src].morphs if morph.pos != "記号"])]
          if len(particles) > 1:
            if len(items) > 1:
              particles = sorted(set(particles))
              items = sorted(set(items))
              particles_form = " ".join(particles)
              items_form = " ".join(items)
              print(f"{morph.base}\t{particles_form}\t{items_form}", file=f)

問４７

In [ ]:
with open("./result47.txt", "w") as f:
  for sentence in sentences:
    for chunk in sentence.chunks:
      for morph in chunk.morphs:
        if morph.pos == "動詞":
          for src in chunk.srcs:
            predicates = []
            if len(sentence.chunks[src].morphs) == 2 and sentence.chunks[src].morphs[0].pos1 == "サ変接続" and sentence.chunks[src].morphs[1].surface == "を":
              predicates = "".join([sentence.chunks[src].morphs[0].surface, sentence.chunks[src].morphs[1].surface, morph.base])
              particles = []
              items = []
              for src in chunk.srcs:
                particles += [morph.surface for morph in sentence.chunks[src].morphs if morph.pos == "助詞"]
                item = "".join([morph.surface for morph in sentence.chunks[src].morphs if morph.pos != "記号"])
                item = item.rstrip()
                if item not in predicates:
                  items.append(item)
              if len(particles) > 1:
                if len(items) > 1:
                  particles = sorted(set(particles))
                  items = sorted(set(items))
                  particles_form = " ".join(particles)
                  items_form = " ".join(items)
                  predicate = " ".join(predicates)
                  print(f"{predicates}\t{particles_form}\t{items_form}", file=f)

問４８

In [ ]:
sentence = sentences[2]
for chunk in sentence.chunks:
  for morph in chunk.morphs:
    if "名詞" in morph.pos:
      path = ["".join(morph.surface for morph in chunk.morphs if morph.pos != "記号")]
      while chunk.dst != -1:
        path.append("".join(morph.surface for morph in sentence.chunks[chunk.dst].morphs if morph.pos != "記号"))
        chunk = sentence.chunks[chunk.dst]
      print("->".join(path))

問４９

In [ ]:
from itertools import combinations
import re

sentence = sentences[2]
nouns = []
for i, chunk in enumerate(sentence.chunks):
  if [morph for morph in chunk.morphs if morph.pos == "名詞"]:
    nouns.append(i)
for i, j in combinations(nouns, 2):
  path_I = []
  path_J = []
  while i != j:
    if i < j: #文節iの構文木経路上に文節jが存在する場合
      path_I.append(i)
      i = sentence.chunks[i].dst
    else: #文節iの構文木経路上に文節jがない場合
      path_J.append(j)
      j = sentence.chunks[j].dst

  if len(path_J) == 0: # 文節Iの構文木上に文節Jが存在する場合
    X = "X" + "".join([morph.surface for morph in sentence.chunks[path_I[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"])
    Y = "Y" +  "".join([morph.surface for morph in sentence.chunks[i].morphs if morph.pos != "名詞" and morph.pos != "記号"])
    chunk_X = re.sub("X+", "X", X)
    chunk_Y = re.sub("Y+", "Y", Y)
    path_ItoJ = [chunk_X] + ["".join(morph.surface for n in path_I[1:] for morph in sentence.chunks[n].morphs)] + [chunk_Y]
    print(" -> ".join(path_ItoJ))
  else: # 文節Iの構文木上に文節Jが存在しない場合
    X = "X" + "".join([morph.surface for morph in sentence.chunks[path_I[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"])
    Y = "Y" + "".join([morph.surface for morph in sentence.chunks[path_J[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"])
    chunk_X = re.sub("X+", "X", X)
    chunk_Y = re.sub("Y+", "Y", Y)
    chunk_k = "".join([morph.surface for morph in sentence.chunks[i].morphs if morph.pos != "記号"])
    path_X = [chunk_X] + ["".join(morph.surface for n in path_I[1:] for morph in sentence.chunks[n].morphs if morph.pos != "記号")]
    path_Y = [chunk_Y] + ["".join(morph.surface for n in path_J[1: ]for morph in sentence.chunks[n].morphs if morph.pos != "記号")]
    print(" | ".join(["->".join(path_X), "->".join(path_Y), chunk_k]))